# imports

In [6]:
import sys
from pathlib import Path
import httpx

# Добавить корень проекта в PYTHONPATH для корректного импорта config.py
current_dir = Path.cwd()
# Если мы в поддиректории notebook/, поднимаемся на уровень выше
if current_dir.name == 'notebook':
    project_root = current_dir.parent
else:
    project_root = current_dir

if project_root.as_posix() not in sys.path:
    print(f"Add to PYTHONPATH: {project_root.as_posix()}")
    sys.path.insert(0, project_root.as_posix())

from config import config

# auth

In [14]:
# It's team client_id, like team025
team_id = "team025"
# It's end client_id, like team025-1
client_id = "team025-1"
# It's team client_secret
client_secret = "jFWNiRoGLuz25TGxpUqmoGcy77VBKCxh"
bank_url = "https://vbank.open.bankingapi.ru"

## get_access_token

In [ ]:
# Get access token from VBank API
async def get_access_token(
    team_id: str, client_secret: str, bank_url: str
) -> dict:
    """
    Obtain bank access token for inter-bank operations.
    
    Args:
        team_id: Team identifier (e.g., "team200")
        client_secret: API key for secure requests
        bank_url: Base URL of the bank API
    
    Returns:
        dict: Token response containing access_token, token_type, client_id, expires_in
    """
    async with httpx.AsyncClient(base_url=bank_url, timeout=30.0) as client:
        response = await client.post(
            "/auth/bank-token",
            params={
                "client_id": team_id,
                "client_secret": client_secret
            }
        )
        response.raise_for_status()
        return response.json()

# Example usage
token_response = await get_access_token(
    team_id, 
    client_secret,
    bank_url,
)
print(f"Access token obtained: {token_response['access_token']}")
print(f"Token type: {token_response['token_type']}")
print(f"Expires in: {token_response['expires_in']} seconds")
access_token = token_response['access_token']

Access token obtained: eyJhbGciOiJIUzI1NiIs...
Token type: bearer
Expires in: 86400 seconds


# consents

## request_account_consent

In [17]:
# Request account access consent
async def request_account_consent(
    access_token: str,
    client_id: str,
    permissions: list[str],
    requesting_bank: str,
    requesting_bank_name: str,
    bank_url: str,
) -> dict:
    """
    Request consent for account access.
    
    Args:
        access_token: Bearer token obtained from get_access_token
        client_id: Client identifier
        permissions: Array of requested scopes (e.g., ['ReadAccountsDetail', 'ReadBalances'])
        requesting_bank: Team ID without client suffix (e.g., 'team200')
        requesting_bank_name: Application display name
        bank_url: Base URL of the bank API
    
    Returns:
        dict: Consent response containing consent_id, status, auto_approved
    """
    async with httpx.AsyncClient(base_url=bank_url, timeout=30.0) as client:
        response = await client.post(
            "/account-consents/request",
            headers={
                "Authorization": f"Bearer {access_token}",
                "Content-Type": "application/json"
            },
            json={
                "client_id": client_id,
                "permissions": permissions,
                "requesting_bank": requesting_bank,
                "requesting_bank_name": requesting_bank_name
            }
        )
        response.raise_for_status()
        return response.json()


permissions = [
    "ReadAccountsDetail", "ReadBalances", "ReadTransactionsDetail",
    "ManageAccounts", "ReadCards", "ManageCards",
]

consent_response = await request_account_consent(
    access_token=access_token,
    client_id=client_id,
    permissions=permissions,
    requesting_bank=team_id,
    requesting_bank_name="Team 025 App",
    bank_url=bank_url,
)

print(f"Consent ID: {consent_response['consent_id']}")
print(f"Status: {consent_response['status']}")
print(f"Auto-approved: {consent_response.get('auto_approved', False)}")
consent_id = consent_response['consent_id']

Consent ID: consent-3b8124834e3f
Status: approved
Auto-approved: True


## get_consent_by_id

In [18]:
# Get consent by ID
async def get_consent_by_id(
    consent_id: str,
    access_token: str,
    bank_url: str,
) -> dict:
    """
    Get consent details by consent ID.
    
    Args:
        consent_id: The consent identifier (e.g., "consent-d8f0724a4775")
        access_token: Bearer token obtained from get_access_token
        bank_url: Base URL of the bank API
    
    Returns:
        dict: Consent data with consentId, status, creationDateTime, permissions, etc.
    """
    async with httpx.AsyncClient(base_url=bank_url, timeout=30.0) as client:
        response = await client.get(
            f"/account-consents/{consent_id}",
            headers={
                "Authorization": f"Bearer {access_token}",
            }
        )
        response.raise_for_status()
        return response.json()


test_consent_id = "consent-3b8124834e3f"
consent_details = await get_consent_by_id(
    consent_id=test_consent_id,
    access_token=access_token,
    bank_url=bank_url,
)

print(f"Consent ID: {consent_details['data']['consentId']}")
print(f"Status: {consent_details['data']['status']}")
print(f"Permissions: {consent_details['data']['permissions']}")
print(f"Created: {consent_details['data']['creationDateTime']}")

Consent ID: consent-3b8124834e3f
Status: Authorized
Permissions: ['ReadAccountsDetail', 'ReadBalances', 'ReadTransactionsDetail', 'ManageAccounts', 'ReadCards', 'ManageCards']
Created: 2025-11-08T19:21:23.373356Z


## revoke_consent

In [19]:
# Revoke consent
async def revoke_consent(
    consent_id: str,
    access_token: str,
    bank_url: str,
) -> bool:
    """
    Revoke consent by consent ID.
    
    Args:
        consent_id: The consent identifier to revoke
        access_token: Bearer token obtained from get_access_token
        bank_url: Base URL of the bank API
    
    Returns:
        bool: True if successfully revoked (HTTP 204)
    """
    async with httpx.AsyncClient(base_url=bank_url, timeout=30.0) as client:
        response = await client.delete(
            f"/account-consents/{consent_id}",
            headers={
                "Authorization": f"Bearer {access_token}",
            }
        )
        response.raise_for_status()
        return response.status_code == 204


revoke_consent_id = "consent-8d5fc700b99f"
is_revoked = await revoke_consent(
    consent_id=revoke_consent_id,
    access_token=access_token,
    bank_url=bank_url,
)

print(f"Consent revoked: {is_revoked}")
print(f"Consent ID {consent_id} status changed to 'Revoked'")

Consent revoked: True
Consent ID consent-3b8124834e3f status changed to 'Revoked'
